 ### BETTER WAY 29. 대입식을 사용해 컴프리헨션 안에서 반복작업을 피하라

In [1]:
stock = {
    '못' : 125,
    '나사못' : 35,
    '나비너트' : 8,
    '와셔' : 24,
}

order = ['나사못', '나비너트', '클립']

In [8]:
def get_batches(count, size):
    return count//size

result = {}

for name in order :
    count = stock.get(name, 0) #  get : 대응되는 키 값을 반환, key값이 없으면 0으로 반환 (원래는 None으로 반환)
    batches = get_batches(count, 8) # count를 받아서 size로 나눈다.
    if batches: # 0이 아니면 True로 넘어감
        result[name] = batches # 다시 dictionary로

In [11]:
found = {name : get_batches(stock.get(name, 0), 8)
         for name in order
         if get_batches(stock.get(name,0), 8)}

print(found)

{'나사못': 4, '나비너트': 1}


문제점 : get_batches(stock.get(name,0), 8)이 반복된다는 단점
> 실수 할 가능성이 높아짐

------
해결 방안
> 왈러스 연산자(:=) 사용  

> 왈러스 연산자 (:=)란?  
    >> 대입과 평가를 한번에 할 수 있는 연산자  

In [12]:
# # 오류 발생 코드
# t = 0
# if (t = 3) != 0:
#     print('True')

# 이때 사용하는 것이 왈러스 연산자
t = 0
if (t := 3) != 0:
    print('True')

True


대입식을 사용하면 stock 딕셔너리에 각 order 키를 한 번만 조회하고 get_batches를 한 번만 호출해서 그 결과를 batches 변수에 저장할 수 있다.

-> 불필요한 연산 감소

In [13]:
found = {name : batches for name in order
        if (batches := get_batches(stock.get(name, 0), 8))}

In [14]:
# # tenth > 0 부분에서 tenth가 정의되어 있지 않으므로 값을 읽을 때 오류 발생
# result = {name (tenth := count // 10)
#           for name, count in stock.items() if tenth > 0} # .items() key와 value를 한꺼번에 for문 반복

# 대입식을 조건 쪽으로 옮기고 대입식에서 만들어진 변수 이름을 컴프리헨션 값 식에서 참조하면 문제 해결
result = {name : tenth for name, count in stock.items()
          if (tenth := count // 10) > 0}
print(result)

{'못': 12, '나사못': 3, '와셔': 2}


In [16]:
# 컴프리헨션이 값 부분에서 왈러스 연산자를 사용할 때, 그 값에 대한 조건부분이 없다면 루프 밖으로 루프 변수 누출

half = [(last := count // 2) for count in stock.values()]
print(f'{half}의 마지막 원소는 {last}')

# for문 루프에서도 비슷
for count in stock.values():
    pass # 클래스나 함수에서 내부 동작은 필요없고, 이름만 전달해주는 경우
print(f'{list(stock.values())}의 마지막 원소는 {count}')

[62, 17, 4, 12]의 마지막 원소는 12
[125, 35, 8, 24]의 마지막 원소는 24


루프 변수가 누출되지 않기 위해서는 컴프리센션 대입식을 조건에만 사용하는 것을 권장

In [17]:
# 바로 앞 예제를 처리하다가 count가 정의 된 경우에는 제대로 작동 X
# 파이썬을 재시작하고 아래 코드를 실행해야 오류 볼 수 있음

half = [count // 2 for count in stock.values()]
print(half) # 작동함
print(count) # 루프 변수가 누출되지 않기 때문에 예외 발생

[62, 17, 4, 12]
24


In [18]:
found = ((name, batches) for name in order
         if (batches := get_batches(stock.get(name, 0), 8)))

print(next(found))
print(next(found))

('나사못', 4)
('나비너트', 1)


### BETTER WAY 30. 리스트를 반환하기보다는 제너레이터를 사용하라

In [24]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

In [36]:
address = '컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어: 전산기)는 진공관'
result = index_words(address)
print(result[:11])

[0, 8, 18, 23, 28, 33, 39]


위 코드의 문제점

1. 코드에 잡음이 많고 핵심을 알아보기 어렵다.
2. 코드는 새로운 결과를 찾을 때 마다 append 메서드를 호출한다.
3. 메서드 호출의 덩어리가 너무 크기 때문에, result.append 리스트에 추가될 값 (index+1)의 중요성을 희석해버린다. >> 무슨말이지?
4. 공백 제회 130여 개 글자가 함수 본문 전체에 들어가는데, 그 중 75개 글자 내외만 중요한 일을 한다.


개선 방안

> 제너레이터
>> 제너레이터란 : Generator란 Iterator를 생성해주는 함수를 의미, Generator는 모든 값을 메모리에 담고 있지 않고, 그때그때 값을 생성해서 반환하기 때문에 제너레이터를 사용할 때는 한 번에 한 개의 값만 순환할 수 있다.
>>> 이터레이션이란 : for 문에서  어떤 객체의 원소에 하나씩 차례로 접근하는 것

yield 식
> 파이썬 제너레이터를 만드는데 사용되는 키워드

In [39]:
# Example 3
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

In [41]:
it = index_words_iter(address)

print(next(it))
print(next(it))
print(next(it))

0
8
18


반환하는 리스트와 상호작용하는 코드가 없어졌으므로 index_words_iter 함수가 훨씬 읽기 쉽다.  
대신 결과는 yield 식에 의해 전달된다.  
제너레이터가 반환하는 이터레이터를 리스트 내장함수에 넘기면, 필요할 때 제너레이터를 쉽게 리스트로 변환할 수 있다.

In [42]:
result = list(index_words_iter(address))
print(result[:10])

[0, 8, 18, 23, 28, 33, 39]


index_words의 두번째 문제점  

반환하기 전에 리스트에 모든 결과를 다 저장해야 한다는 것  
> 입력이 매우 크면 프로그램이 메모리를 소진해서 중단될 수 있음.
>> 같은 함수를 제너레이터 버전으로 만들면 사용하는 메모리 크기를 어느정도 제한할 수 있으므로 입력 길이가 아무리 길어도 쉽게 처리할 수 있다.  

In [60]:
# 함수의 작업 메모리는 입력 중 가장 긴 줄의 길이로 제한

def index_file(handle):
    offset = 0 # offset == index
    for line in handle :
        if line:
            yield offset
        for letter in line :
            offset += 1
            if letter == ' ' :
                yield offset # yield : 제너레이터 값을 전환하고 반환

In [68]:
import itertools

with open('address.txt','r', encoding='utf-8') as f :
    it = index_file(f)
    results = itertools.islice(it, 0, 10) # intertools : 효율적인 루핑을 위한 이터레이터를 만드는 함수, islice : 0번부터 10개까지 잘라라
    print(list(results))

[0, 8, 18, 23, 28, 33, 39, 43, 52, 64]


### BEETER WAY 31 : 인자에 대해 이터레이션 할 때는 방어적이 돼라

객체가 원소로 들어있는 리스트를 함수가 파라미터로 받았을 때, 리스트를 여러 번 이터레이션 하는 것이 중요할 때가 종종 있다.

- 미국 텍사스 주의 여행자수 분석하고 싶음
- 데이터 집합이 도시별 방문자 수라고 가정
- 각 도시가 전체 여행자 수 중에 차지하는 비율을 계산하고 싶음

-> 1년 전체 여행자 수를 계산하기 위해 입력 전체 합계를 내고, 이 합계로 각 도시의 방문자 수를 나누는 정규화 함수 필요

In [70]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [71]:
visits = [15,35,80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

# assert : 주어진 조건이 True가 아니면 AssertionError를 발생시키는 역할
# assert 조건, 에러메세지
# assert 역할 : 코드 내 특정 조건을 검증하고, 코드의 안정성과 디버깅을 도와주는 역할

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [72]:
def read_visits(data_path):
    with open(data_path) as f: # with문을 사용하면 파일을 열고 닫는 과정을 자동으로 처리할 수 있음 / 파일 객체 f에 할당 -> f를 이용하여 파일 읽거나 쓰기 가능
        for line in f:
            yield int(line)

In [77]:
it = read_visits('my_numbers.txt') # 여기서 이미 생성됨
percentages = normalize(it) # 다시 이터레이션
print(percentages)

<generator object read_visits at 0x7f3018109a80>
[]


아무 결과도 나오지 않은 이유  

이터레이터가 결과를 단 한 번만 만들어내기 때문에, StopIteration 예외가 발생한 이터레이터나 제너레이터를 다시 이터레이션 하면 아무 결과도 얻을 수 X

In [76]:
it = read_visits('my_numbers.txt')
print(list(it))
print(list(it))

[15, 35, 80]
[]


for loop, list 생성자, 파이썬 표준 라이브러리에 있는 많은 함수가 일반적인 연산 도중에 StopIteration 예외가 던져지는 것을 가정.
> 이런 함수들은 출력이 없는 이터레이터와 이미 소진돼버린 이터레이터를 구분할 수 X

In [ ]:
def normalize_copy(numbers) :
    numbers_copy = list(numbers) # 이터레이터 복사
    total = sum(numbers_copy)
    result = []
    for value in numbers_copy :
        percent = 100 * value / total
        result.append(percent)
    return result